GLiNER is a Named Entity Recognition (NER) model capable of identifying any entity type using a bidirectional transformer encoder (BERT-like). It provides a practical alternative to traditional NER models, which are limited to predefined entities, and Large Language Models (LLMs) that, despite their flexibility, are costly and large for resource-constrained scenarios.

https://github.com/urchade/GLiNER <br>
https://huggingface.co/urchade/gliner_medium-v2.1

In [ ]:
import pandas as pd
from gliner import GLiNER

In [7]:
model = GLiNER.from_pretrained("urchade/gliner_medium")
model.eval()
print("ok")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/4.80k [00:00<?, ?B/s]

c:\Users\racha\Desktop\PROJECTS\PKO\analysis-advanced-chunking-techniques\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\racha\.cache\huggingface\hub\models--urchade--gliner_medium. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/781M [00:00<?, ?B/s]

c:\Users\racha\Desktop\PROJECTS\PKO\analysis-advanced-chunking-techniques\.venv\Lib\site-packages\transformers\convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


ok


Threshold w GLiNER (ang. confidence threshold) to minimalna wartość pewności modelu, powyżej której prognoza (dana etykieta) zostanie uznana za prawidłową i zwrócona w wynikach.

In [9]:
text = """
1. Użyte w Regulaminie określenia oznaczają:
1) dopuszczalne saldo debetowe – ujemne saldo rachunku oszczędnościowo-rozliczeniowego w wysokości nieprzekraczającej miesięcznego limitu ustalanego
przez PKO Bank Polski SA, z którego Posiadacz rachunku może korzystać na podstawie umowy rachunku,
2) data waluty – dzień, od którego lub do którego, naliczane są odsetki od środków pieniężnych, którymi został obciążony lub uznany rachunek,
3) dyspozycja – wszelkie oświadczenia woli lub wiedzy, w tym dotyczące operacji, składane przez Klienta,
4) dzień roboczy – dzień przypadający od poniedziałku do piątku, który nie jest dniem ustawowo wolnym od pracy,
5) kapitalizacja – dopisanie do salda rachunku naliczonych za dany okres odsetek od środków pieniężnych zgromadzonych na tym rachunku
"""

labels = [
    "kredyt hipoteczny", "pożyczka gotówkowa", "konto osobiste", "konto firmowe", "konto oszczędnościowe", "lokata terminowa",
    "ubezpieczenie", "kredyt firmowy", "kredyt konsolidacyjny", "karta kredytowa", "karta debetowa", "bankomat",
    "oddział bankowy", "opłaty", "prowizja", "transakcje", "limit kredytowy", "umowa kredytowa", "oprocentowanie",
    "kapitał", "odsetki", "raty", "czyszczenie historii kredytowej", "ubezpieczenie nieruchomości", "ubezpieczenie na życie",

]

entities = model.predict_entities(text, labels, threshold=0.4, multi_label=True)

for entity in entities:
    print(entity["text"], "=>", entity["label"])

Posiadacz rachunku => konto osobiste
umowy rachunku => umowa kredytowa
środków pieniężnych => odsetki


In [27]:
chunks = [
"""
1. Użyte w Regulaminie określenia oznaczają:
1) dopuszczalne saldo debetowe – ujemne saldo rachunku oszczędnościowo-rozliczeniowego w wysokości nieprzekraczającej miesięcznego limitu ustalanego
przez PKO Bank Polski SA, z którego Posiadacz rachunku może korzystać na podstawie umowy rachunku,
2) data waluty – dzień, od którego lub do którego, naliczane są odsetki od środków pieniężnych, którymi został obciążony lub uznany rachunek,
3) dyspozycja – wszelkie oświadczenia woli lub wiedzy, w tym dotyczące operacji, składane przez Klienta,
4) dzień roboczy – dzień przypadający od poniedziałku do piątku, który nie jest dniem ustawowo wolnym od pracy,
5) kapitalizacja – dopisanie do salda rachunku naliczonych za dany okres odsetek od środków pieniężnych zgromadzonych na tym rachunku
""",
"""
1. PKO Bank Polski SA może prowadzić na rzecz Klientów rachunki bankowe, o których mowa w § 1. Rachunki te nie są przeznaczone do przeprowadzania rozliczeń
pieniężnych związanych z prowadzoną przez Klienta działalnością gospodarczą lub zarobkową.
2. PKO Bank Polski SA może otwierać kolejne rachunki na podstawie umowy o prowadzenie rachunków bankowych.
3. Zawarcie umowy rachunku może nastąpić:
1) za pośrednictwem elektronicznych kanałów dostępu, o ile PKO Bank Polski SA dla danego kanału udostępnia odpowiednią funkcjonalność,
2) w placówce,
3) w inny uzgodniony sposób.
4. Rachunek może być otwarty:
1) jako rachunek indywidualny - dla rezydenta albo nierezydenta,
2) jako rachunek wspólny – dla dwóch osób o tym samym statusie dewizowym, posiadających pełną zdolność do czynności prawnych, które posiadają ten
sam kraj rezydencji podatkowej i obie legitymują się ważnymi certyfikatami rezydencji podatkowej, bądź obie nie posiadają ważnych certyfikatów rezydencji
podatkowej
""",
"""
1. Posiadacz rachunku, który nie ukończył 13 lat, albo osoba ubezwłasnowolniona całkowicie nie może samodzielnie dysponować środkami pieniężnymi
zgromadzonymi na rachunku, ani dokonywać samodzielnie żadnych czynności związanych z posiadaniem rachunku.
2. W imieniu Posiadacza rachunku, o którym mowa w ust. 1, do dysponowania środkami pieniężnymi zgromadzonymi na rachunku w granicach zwykłego zarządu,
uprawniony jest jego przedstawiciel ustawowy, który zawarł umowę rachunku w imieniu Posiadacza rachunku. Dysponowanie środkami pieniężnymi w zakresie
przekraczającym zwykły zarząd, wymaga zezwolenia sądu opiekuńczego
""",
"""
1. Za usługi świadczone przez PKO Bank Polski SA, w tym za czynności związane z prowadzeniem rachunku, wysyłaniem powiadomień, a także za inne czynności,
PKO Bank Polski SA pobiera prowizje i opłaty bankowe, zgodnie z Taryfą.
2. PKO Bank Polski SA należne prowizje i opłaty bankowe, o których mowa w ust. 1, pobierać będzie, z uwzględnieniem ust. 3-4, w ciężar rachunku, za wyjątkiem
rachunku lokaty.
3. W przypadku rachunku lokaty należne prowizje i opłaty bankowe PKO Bank Polski SA pobierać będzie z rachunku, o którym mowa w § 25 ust. 2 albo w formie
gotówkowej.
4. W przypadku braku wolnych środków na rachunku, pobranie:
1) prowizji i opłat, lub
2) zaliczek na podatek dochodowy oraz składek na ubezpieczenie zdrowotne od realizowanych przez PKO Bank Polski SA rent i emerytur z zagranicy
może spowodować powstanie zadłużenia przeterminowanego.
""",
"""
1. Środki pieniężne zgromadzone na rachunku mogą być oprocentowane zgodnie z zasadami określonymi w Regulaminie lub w umowie rachunku.
2. W przypadku, gdy środki pieniężne zgromadzone na rachunku są oprocentowane według zmiennej stopy procentowej, w okresie obowiązywania umowy
rachunku, PKO Bank Polski SA jest uprawniony do zmiany wysokości oprocentowania środków pieniężnych zgromadzonych na tym rachunku na zasadach
określonych w ust. 3.
""",
"""Oddział naszego banku zlokalizowana przy ulicy Warszawskiej 12 oferuje pełen zakres usług finansowych dla klientów indywidualnych i firmowych.
Klienci mogą tam skorzystać z doradztwa kredytowego oraz założyć rachunek oszczędnościowy.
""",
"""
Oddział bankowy zyskał na znaczeniu!
"""
]

In [28]:
entities = model.batch_predict_entities(chunks, labels, multi_label=True)

In [29]:
entities

[[],
 [],
 [{'start': 43,
   'end': 49,
   'text': '13 lat',
   'label': 'limit kredytowy',
   'score': 0.7323434948921204},
  {'start': 277,
   'end': 285,
   'text': 'rachunku',
   'label': 'konto osobiste',
   'score': 0.5565055012702942},
  {'start': 499,
   'end': 507,
   'text': 'rachunku',
   'label': 'konto osobiste',
   'score': 0.5390024781227112}],
 [{'start': 182,
   'end': 190,
   'text': 'prowizje',
   'label': 'prowizja',
   'score': 0.7285996079444885},
  {'start': 182,
   'end': 190,
   'text': 'prowizje',
   'label': 'opłaty',
   'score': 0.6407319903373718},
  {'start': 257,
   'end': 265,
   'text': 'prowizje',
   'label': 'prowizja',
   'score': 0.6908012628555298},
  {'start': 257,
   'end': 265,
   'text': 'prowizje',
   'label': 'opłaty',
   'score': 0.644463300704956},
  {'start': 441,
   'end': 449,
   'text': 'prowizje',
   'label': 'prowizja',
   'score': 0.6739863753318787},
  {'start': 441,
   'end': 449,
   'text': 'prowizje',
   'label': 'opłaty',
   'sc

In [30]:
results = []
for chunk, entity_list in zip(chunks, entities):
    matched_labels = list(set(entity['label'] for entity in entity_list)) 
    results.append({"Chunk": chunk, "Matched Labels": matched_labels})

# Konwersja do DataFrame
results_df = pd.DataFrame(results)

# Wyświetlenie tabeli
results_df

,Chunk,Matched Labels
0,\n1. Użyte w Regulaminie określenia oznaczają:...,[]
1,\n1. PKO Bank Polski SA może prowadzić na rzec...,[]
2,"\n1. Posiadacz rachunku, który nie ukończył 13...","[konto osobiste, limit kredytowy]"
3,\n1. Za usługi świadczone przez PKO Bank Polsk...,"[opłaty, prowizja]"
4,\n1. Środki pieniężne zgromadzone na rachunku ...,"[umowa kredytowa, konto osobiste]"
5,Oddział naszego banku zlokalizowana przy ulicy...,[]
6,\nOddział bankowy zyskał na znaczeniu!\n,[oddział bankowy]
